In [10]:
# ========================================
# AFib Detection Model - 99% Accuracy Target
# Neural Network + XGBoost Ensemble
# Labels: 1=AFib, 0=Normal
# ========================================

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, IsolationForest
from sklearn.metrics import accuracy_score, confusion_matrix, roc_auc_score, f1_score
from sklearn.feature_selection import SelectKBest, f_classif, RFE
from imblearn.over_sampling import BorderlineSMOTE
import tensorflow as tf
from tensorflow.keras import layers, Model, callbacks, regularizers
import xgboost as xgb
import optuna
import os
import json
from joblib import dump, load
import warnings
warnings.filterwarnings('ignore')

# Set random seed for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

# =============================================================================
# Data Pipeline for 99% Accuracy
# =============================================================================

def load_and_prepare_data():
    """Clean data preparation targeting 99% accuracy."""
    print("Loading and preparing data for 99% accuracy target...")
    
    # Load data
    features = pd.read_csv('features_output1.csv', header=None)
    labels = pd.read_csv('label.csv', header=None)
    
    # Filter and convert labels
    mask = labels[0].isin([1, 2])
    X = features.loc[mask].values
    y = np.where(labels.loc[mask, 0] == 2, 1, 0)
    
    print(f"Original dataset: {X.shape}, AFib: {np.sum(y==1)}, Normal: {np.sum(y==0)}")
    
    # Remove outliers using Isolation Forest
    iso_forest = IsolationForest(contamination=0.1, random_state=42)
    outlier_mask = iso_forest.fit_predict(X) == 1
    X_clean = X[outlier_mask]
    y_clean = y[outlier_mask]
    
    print(f"After outlier removal: {X_clean.shape}, Removed: {np.sum(~outlier_mask)} samples")
    
    # Split data
    X_temp, X_test, y_temp, y_test = train_test_split(
        X_clean, y_clean, test_size=0.15, stratify=y_clean, random_state=42
    )
    X_train, X_val, y_train, y_val = train_test_split(
        X_temp, y_temp, test_size=0.15, stratify=y_temp, random_state=42
    )
    
    # Handle missing values
    train_df = pd.DataFrame(X_train)
    if train_df.isna().sum().sum() > 0:
        train_means = train_df.mean()
        X_train = train_df.fillna(train_means).values
        X_val = pd.DataFrame(X_val).fillna(train_means).values
        X_test = pd.DataFrame(X_test).fillna(train_means).values
    
    # Robust scaling
    scaler = RobustScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_val_scaled = scaler.transform(X_val)
    X_test_scaled = scaler.transform(X_test)
    
    # Advanced feature selection using consensus approach
    print("Advanced feature selection for maximum discrimination...")
    
    # Method 1: Statistical selection
    selector_stats = SelectKBest(score_func=f_classif, k=30)
    selector_stats.fit(X_train_scaled, y_train)
    stats_features = set(selector_stats.get_support(indices=True))
    
    # Method 2: Recursive Feature Elimination  
    rf_rfe = RandomForestClassifier(n_estimators=200, random_state=42)
    selector_rfe = RFE(rf_rfe, n_features_to_select=30, step=1)
    selector_rfe.fit(X_train_scaled, y_train)
    rfe_features = set(selector_rfe.get_support(indices=True))
    
    # Method 3: Tree-based importance
    et_selector = ExtraTreesClassifier(n_estimators=200, random_state=42)
    et_selector.fit(X_train_scaled, y_train)
    top_tree_features = set(np.argsort(et_selector.feature_importances_)[-30:])
    
    # Take consensus features (intersection for highest confidence)
    consensus_features = list(stats_features & rfe_features & top_tree_features)
    
    if len(consensus_features) < 15:
        # If intersection too small, take union of top features
        all_features = stats_features | rfe_features | top_tree_features
        consensus_features = list(all_features)[:25]
    
    print(f"Consensus features selected: {len(consensus_features)}")
    
    X_train_selected = X_train_scaled[:, consensus_features]
    X_val_selected = X_val_scaled[:, consensus_features]
    X_test_selected = X_test_scaled[:, consensus_features]
    
    # Conservative class balancing for accuracy preservation
    print("Conservative class balancing for accuracy preservation...")
    balancer = BorderlineSMOTE(random_state=42, k_neighbors=3, kind='borderline-1')
    X_train_balanced, y_train_balanced = balancer.fit_resample(X_train_selected, y_train)
    
    # Limit oversampling to maintain accuracy
    afib_count = np.sum(y_train_balanced == 1)
    normal_count = np.sum(y_train_balanced == 0)
    
    if afib_count > normal_count * 0.7:
        target_afib = int(normal_count * 0.7)
        afib_indices = np.where(y_train_balanced == 1)[0]
        keep_afib_indices = np.random.choice(afib_indices, target_afib, replace=False)
        normal_indices = np.where(y_train_balanced == 0)[0]
        
        keep_indices = np.concatenate([normal_indices, keep_afib_indices])
        X_train_balanced = X_train_balanced[keep_indices]
        y_train_balanced = y_train_balanced[keep_indices]
    
    print(f"Final balanced dataset - AFib: {np.sum(y_train_balanced==1)}, Normal: {np.sum(y_train_balanced==0)}")
    
    return {
        'train': (X_train_balanced, y_train_balanced),
        'val': (X_val_selected, y_val),
        'test': (X_test_selected, y_test),
        'scaler': scaler,
        'features': np.array(consensus_features)
    }

# =============================================================================
# Precision-Optimized Neural Network
# =============================================================================

class BalancedNeuralNetwork:
    """Neural network optimized for balanced performance (95% accuracy + 70% sensitivity)."""
    
    def __init__(self, input_dim):
        self.input_dim = input_dim
        self.model = self._build_balanced_model()
    
    def _build_balanced_model(self):
        """Build model optimized for balanced performance."""
        inputs = layers.Input(shape=(self.input_dim,))
        
        # Moderate regularization for balance
        x = layers.Dense(256, activation='relu', 
                        kernel_regularizer=regularizers.l1_l2(0.005, 0.005))(inputs)
        x = layers.BatchNormalization()(x)
        x = layers.Dropout(0.3)(x)
        
        x = layers.Dense(128, activation='relu',
                        kernel_regularizer=regularizers.l1_l2(0.005, 0.005))(x)
        x = layers.BatchNormalization()(x)
        x = layers.Dropout(0.3)(x)
        
        x = layers.Dense(64, activation='relu',
                        kernel_regularizer=regularizers.l1_l2(0.005, 0.005))(x)
        x = layers.Dropout(0.2)(x)
        
        # Balanced output layer
        outputs = layers.Dense(1, activation='sigmoid',
                             kernel_regularizer=regularizers.l2(0.005))(x)
        
        model = Model(inputs, outputs)
        model.compile(
            optimizer=tf.keras.optimizers.Adam(learning_rate=0.0005),
            loss=self._balanced_loss,
            metrics=['accuracy']
        )
        return model
    
    @staticmethod
    def _balanced_loss(y_true, y_pred):
        """Balanced loss function for accuracy and sensitivity."""
        bce = tf.keras.losses.binary_crossentropy(y_true, y_pred)
        # Moderate penalty for both false positives and false negatives
        false_positive_penalty = 2 * (1 - y_true) * y_pred
        false_negative_penalty = 3 * y_true * (1 - y_pred)
        return bce + false_positive_penalty + false_negative_penalty
    
    def fit(self, X_train, y_train, X_val, y_val):
        """Train with balanced regularization."""
        callbacks_list = [
            callbacks.EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True),
            callbacks.ReduceLROnPlateau(monitor='val_loss', patience=8, factor=0.5, min_lr=1e-6)
        ]
        
        self.model.fit(
            X_train, y_train,
            validation_data=(X_val, y_val),
            epochs=150,
            batch_size=64,
            callbacks=callbacks_list,
            verbose=0
        )
    
    def predict_proba(self, X):
        """Predict probabilities."""
        probs = self.model.predict(X, verbose=0)
        return np.column_stack([1 - probs, probs])

# =============================================================================
# Ultra-Conservative XGBoost
# =============================================================================

def create_balanced_xgboost(X_train, y_train, X_val, y_val):
    """Create XGBoost optimized for balanced performance (95% accuracy + 70% sensitivity)."""
    print("Optimizing XGBoost for balanced performance...")
    
    def objective(trial):
        params = {
            'n_estimators': trial.suggest_int('n_estimators', 200, 600),
            'max_depth': trial.suggest_int('max_depth', 4, 10),
            'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.15),
            'subsample': trial.suggest_float('subsample', 0.7, 1.0),
            'colsample_bytree': trial.suggest_float('colsample_bytree', 0.7, 1.0),
            'reg_alpha': trial.suggest_float('reg_alpha', 0.1, 2.0),
            'reg_lambda': trial.suggest_float('reg_lambda', 0.1, 2.0),
            'scale_pos_weight': trial.suggest_float('scale_pos_weight', 2.0, 8.0),
            'random_state': 42,
            'verbosity': 0
        }
        
        model = xgb.XGBClassifier(**params)
        model.fit(X_train, y_train)
        y_pred_proba = model.predict_proba(X_val)[:, 1]
        
        # Optimize for balanced performance
        best_score = 0
        for threshold in np.arange(0.3, 0.8, 0.05):
            y_pred = (y_pred_proba >= threshold).astype(int)
            accuracy = accuracy_score(y_val, y_pred)
            
            cm = confusion_matrix(y_val, y_pred)
            if cm.shape == (2, 2):
                tn, fp, fn, tp = cm.ravel()
                sensitivity = tp / (tp + fn) if (tp + fn) > 0 else 0
                
                # Balanced scoring: 60% accuracy + 40% sensitivity
                accuracy_score_norm = min(accuracy / 0.95, 1.0)
                sensitivity_score_norm = min(sensitivity / 0.70, 1.0)
                balanced_score = 0.6 * accuracy_score_norm + 0.4 * sensitivity_score_norm
                
                best_score = max(best_score, balanced_score)
        
        return best_score
    
    study = optuna.create_study(direction='maximize')
    study.optimize(objective, n_trials=50, show_progress_bar=False)
    
    print(f"Best XGBoost balanced score: {study.best_value:.4f}")
    
    best_xgb = xgb.XGBClassifier(**study.best_params)
    best_xgb.fit(X_train, y_train)
    return best_xgb

# =============================================================================
# Precision Ensemble (Neural Network + XGBoost)
# =============================================================================

class BalancedEnsemble:
    """Two-model ensemble optimized for balanced performance (95% accuracy + 70% sensitivity)."""
    
    def __init__(self, input_dim):
        self.input_dim = input_dim
        self.neural_net = None
        self.xgboost = None
        self.weights = {'neural_net': 0.5, 'xgboost': 0.5}
        
    def fit(self, X_train, y_train, X_val, y_val):
        """Train both models and optimize ensemble weights."""
        print("Training Balanced Neural Network...")
        self.neural_net = BalancedNeuralNetwork(self.input_dim)
        self.neural_net.fit(X_train, y_train, X_val, y_val)
        
        print("Training Balanced XGBoost...")
        self.xgboost = create_balanced_xgboost(X_train, y_train, X_val, y_val)
        
        print("Optimizing ensemble weights for balanced performance...")
        self._optimize_ensemble_weights(X_val, y_val)
    
    def _optimize_ensemble_weights(self, X_val, y_val):
        """Optimize ensemble weights for balanced performance."""
        nn_probs = self.neural_net.predict_proba(X_val)[:, 1]
        xgb_probs = self.xgboost.predict_proba(X_val)[:, 1]
        
        def objective(trial):
            weight_nn = trial.suggest_float('weight_nn', 0.1, 0.9)
            weight_xgb = 1 - weight_nn
            
            ensemble_pred = weight_nn * nn_probs + weight_xgb * xgb_probs
            
            # Find best threshold for balanced performance
            best_score = 0
            for threshold in np.arange(0.3, 0.8, 0.02):
                y_pred = (ensemble_pred >= threshold).astype(int)
                accuracy = accuracy_score(y_val, y_pred)
                
                cm = confusion_matrix(y_val, y_pred)
                if cm.shape == (2, 2):
                    tn, fp, fn, tp = cm.ravel()
                    sensitivity = tp / (tp + fn) if (tp + fn) > 0 else 0
                    
                    # Balanced scoring
                    accuracy_score_norm = min(accuracy / 0.95, 1.0)
                    sensitivity_score_norm = min(sensitivity / 0.70, 1.0)
                    balanced_score = 0.6 * accuracy_score_norm + 0.4 * sensitivity_score_norm
                    
                    best_score = max(best_score, balanced_score)
            
            return best_score
        
        study = optuna.create_study(direction='maximize')
        study.optimize(objective, n_trials=50, show_progress_bar=False)
        
        optimal_nn_weight = study.best_params['weight_nn']
        self.weights = {
            'neural_net': optimal_nn_weight,
            'xgboost': 1 - optimal_nn_weight
        }
        
        print(f"Optimal ensemble weights - Neural Net: {self.weights['neural_net']:.3f}, XGBoost: {self.weights['xgboost']:.3f}")
    
    def predict_proba(self, X):
        """Ensemble prediction for balanced performance."""
        nn_probs = self.neural_net.predict_proba(X)[:, 1]
        xgb_probs = self.xgboost.predict_proba(X)[:, 1]
        
        ensemble_probs = (self.weights['neural_net'] * nn_probs + 
                         self.weights['xgboost'] * xgb_probs)
        
        return np.column_stack([1 - ensemble_probs, ensemble_probs])

# =============================================================================
# Threshold Optimization for 99% Accuracy
# =============================================================================

def optimize_for_balanced_performance(model, X_val, y_val):
    """Find threshold that balances 95% accuracy target with 70% sensitivity."""
    val_probs = model.predict_proba(X_val)[:, 1]
    
    results = []
    print("\nThreshold analysis for balanced performance (95% accuracy + 70% sensitivity):")
    print("Threshold  Accuracy  Sensitivity  Specificity   FN   FP  Score")
    print("-" * 65)
    
    best_threshold = 0.5
    best_score = 0
    
    for threshold in np.arange(0.3, 0.8, 0.02):
        preds = (val_probs >= threshold).astype(int)
        cm = confusion_matrix(y_val, preds)
        
        if cm.shape == (2, 2):
            tn, fp, fn, tp = cm.ravel()
            accuracy = (tp + tn) / len(y_val)
            sensitivity = tp / (tp + fn) if (tp + fn) > 0 else 0
            specificity = tn / (tn + fp) if (tn + fp) > 0 else 0
            
            # Balanced scoring: prioritize both accuracy and sensitivity
            # Give bonus for reaching targets: 95% accuracy, 70% sensitivity
            accuracy_score = min(accuracy / 0.95, 1.0)  # Normalize to 95% target
            sensitivity_score = min(sensitivity / 0.70, 1.0)  # Normalize to 70% target
            
            # Combined score: 60% accuracy + 40% sensitivity
            combined_score = 0.6 * accuracy_score + 0.4 * sensitivity_score
            
            results.append((threshold, accuracy, sensitivity, specificity, fn, fp, combined_score))
            
            # Highlight good balanced performance
            marker = ">>>" if accuracy >= 0.93 and sensitivity >= 0.65 else "   "
            print(f"{marker} {threshold:.3f}    {accuracy:.4f}     {sensitivity:.4f}      {specificity:.4f}    {fn:2d}  {fp:2d}  {combined_score:.3f}")
            
            if combined_score > best_score:
                best_score = combined_score
                best_threshold = threshold
    
    # Find the result for best threshold
    best_result = None
    for result in results:
        if abs(result[0] - best_threshold) < 0.001:
            best_result = result
            break
    
    if best_result:
        _, accuracy, sensitivity, _, _, _, _ = best_result
        print(f"\nSelected threshold: {best_threshold:.3f}")
        print(f"Expected accuracy: {accuracy:.4f} ({accuracy*100:.2f}%)")
        print(f"Expected sensitivity: {sensitivity:.4f} ({sensitivity*100:.1f}%)")
        print(f"Balanced score: {best_score:.3f}")
    
    return best_threshold

# =============================================================================
# Model Evaluation
# =============================================================================

def evaluate_balanced_model(model, X_test, y_test, threshold):
    """Comprehensive evaluation for balanced performance target (95% accuracy + 70% sensitivity)."""
    test_probs = model.predict_proba(X_test)[:, 1]
    test_preds = (test_probs >= threshold).astype(int)
    
    # Calculate all metrics
    accuracy = accuracy_score(y_test, test_preds)
    auc_score = roc_auc_score(y_test, test_probs)
    f1 = f1_score(y_test, test_preds)
    
    cm = confusion_matrix(y_test, test_preds)
    tn, fp, fn, tp = cm.ravel()
    
    sensitivity = tp / (tp + fn) if (tp + fn) > 0 else 0
    specificity = tn / (tn + fp) if (tn + fp) > 0 else 0
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    npv = tn / (tn + fn) if (tn + fn) > 0 else 0
    
    results = {
        'accuracy': accuracy, 'auc': auc_score, 'f1': f1,
        'sensitivity': sensitivity, 'specificity': specificity, 'precision': precision,
        'npv': npv, 'fn': fn, 'fp': fp, 'threshold': threshold
    }
    
    print("\n" + "="*60)
    print("BALANCED MODEL RESULTS - 95% ACCURACY + 70% SENSITIVITY TARGET")
    print("="*60)
    print(f"Accuracy:     {accuracy:.4f} ({accuracy*100:.2f}%)")
    print(f"Sensitivity:  {sensitivity:.4f} ({sensitivity*100:.1f}%)")
    print(f"Specificity:  {specificity:.4f} ({specificity*100:.1f}%)")
    print(f"Precision:    {precision:.4f} ({precision*100:.1f}%)")
    print(f"NPV:          {npv:.4f} ({npv*100:.1f}%)")
    print(f"AUC:          {auc_score:.4f}")
    print(f"F1-Score:     {f1:.4f}")
    
    print(f"\nError Analysis:")
    print(f"Total Test Samples: {len(y_test)}")
    print(f"Correct Predictions: {tp + tn}")
    print(f"Total Errors: {fp + fn} ({((fp + fn)/len(y_test)*100):.1f}%)")
    print(f"False Positives: {fp} (Normal predicted as AFib)")
    print(f"False Negatives: {fn} (AFib predicted as Normal)")
    
    # Balanced performance assessment
    print(f"\nBalanced Performance Target Assessment:")
    accuracy_target_met = accuracy >= 0.95
    sensitivity_target_met = sensitivity >= 0.70
    
    if accuracy_target_met and sensitivity_target_met:
        print("EXCELLENT: Both targets achieved!")
        print(f"  Accuracy: {accuracy*100:.1f}% (target: 95%+)")
        print(f"  Sensitivity: {sensitivity*100:.1f}% (target: 70%+)")
    elif accuracy >= 0.92 and sensitivity >= 0.65:
        print("VERY GOOD: Close to both targets")
        print(f"  Accuracy: {accuracy*100:.1f}% (target: 95%, gap: {(0.95-accuracy)*100:.1f}%)")
        print(f"  Sensitivity: {sensitivity*100:.1f}% (target: 70%, gap: {(0.70-sensitivity)*100:.1f}%)")
    elif accuracy >= 0.90 and sensitivity >= 0.60:
        print("GOOD: Acceptable balanced performance")
        print(f"  Accuracy: {accuracy*100:.1f}% (target: 95%, gap: {(0.95-accuracy)*100:.1f}%)")
        print(f"  Sensitivity: {sensitivity*100:.1f}% (target: 70%, gap: {(0.70-sensitivity)*100:.1f}%)")
    else:
        print("NEEDS IMPROVEMENT: Below balanced performance targets")
        print(f"  Accuracy: {accuracy*100:.1f}% (target: 95%, gap: {(0.95-accuracy)*100:.1f}%)")
        print(f"  Sensitivity: {sensitivity*100:.1f}% (target: 70%, gap: {max(0, 0.70-sensitivity)*100:.1f}%)")
    
    # Clinical deployment assessment
    print(f"\nClinical Deployment Assessment:")
    if accuracy >= 0.93 and sensitivity >= 0.65:
        print("READY FOR DEPLOYMENT: Excellent medical-grade performance")
        print("  Suitable for primary AFib screening with minimal human oversight")
    elif accuracy >= 0.90 and sensitivity >= 0.60:
        print("READY FOR DEPLOYMENT: Good medical-grade performance")
        print("  Suitable for AFib screening with standard clinical workflow")
    elif accuracy >= 0.88 and sensitivity >= 0.55:
        print("CLINICAL REVIEW NEEDED: Acceptable but needs workflow design")
        print("  Recommend: AI screening + mandatory cardiologist review")
    else:
        print("NOT READY: Requires model improvement or restricted use case")
    
    # Comparison with baseline
    baseline_accuracy = 0.8886
    baseline_sensitivity = 0.5135
    baseline_fn = 72
    
    print(f"\nImprovement over baseline:")
    print(f"Accuracy: {accuracy:.4f} vs {baseline_accuracy:.4f} ({(accuracy-baseline_accuracy)*100:+.2f}%)")
    print(f"Sensitivity: {sensitivity:.4f} vs {baseline_sensitivity:.4f} ({(sensitivity-baseline_sensitivity)*100:+.2f}%)")
    print(f"Missed AFib: {fn} vs {baseline_fn} ({fn-baseline_fn:+d} cases)")
    
    # Calculate balanced score
    accuracy_score_norm = min(accuracy / 0.95, 1.0)
    sensitivity_score_norm = min(sensitivity / 0.70, 1.0)
    balanced_score = 0.6 * accuracy_score_norm + 0.4 * sensitivity_score_norm
    
    print(f"\nBalanced Performance Score: {balanced_score:.3f}/1.000")
    
    return results

# =============================================================================
# Model Persistence
# =============================================================================

def save_balanced_model(ensemble, scaler, features, threshold, results, model_dir='balanced_models'):
    """Save balanced performance model components."""
    os.makedirs(model_dir, exist_ok=True)
    
    # Save model components
    ensemble.neural_net.model.save(f'{model_dir}/neural_network.keras')
    dump(ensemble.xgboost, f'{model_dir}/xgboost.joblib')
    dump(scaler, f'{model_dir}/scaler.joblib')
    
    # Convert numpy types for JSON serialization
    def convert_numpy_types(obj):
        if isinstance(obj, np.integer):
            return int(obj)
        elif isinstance(obj, np.floating):
            return float(obj)
        elif isinstance(obj, np.ndarray):
            return obj.tolist()
        elif isinstance(obj, dict):
            return {k: convert_numpy_types(v) for k, v in obj.items()}
        elif isinstance(obj, list):
            return [convert_numpy_types(item) for item in obj]
        return obj
    
    # Save metadata
    metadata = {
        'model_type': 'balanced_ensemble_neural_network_xgboost',
        'label_mapping': {1: 'AFib', 0: 'Normal'},
        'threshold': float(threshold),
        'selected_features': features.tolist(),
        'ensemble_weights': convert_numpy_types(ensemble.weights),
        'performance': convert_numpy_types(results),
        'target_accuracy': 0.95,
        'target_sensitivity': 0.70
    }
    
    with open(f'{model_dir}/metadata.json', 'w') as f:
        json.dump(metadata, f, indent=2)
    
    print(f"Balanced model saved to: {model_dir}/")

class BalancedAFibDetector:
    """Production deployment class for balanced AFib detection."""
    
    def __init__(self, model_dir='balanced_models'):
        """Load balanced model components."""
        print("Loading balanced AFib detection model...")
        
        # Load models
        self.neural_net = tf.keras.models.load_model(
            f'{model_dir}/neural_network.keras',
            custom_objects={'_balanced_loss': BalancedNeuralNetwork._balanced_loss}
        )
        self.xgboost = load(f'{model_dir}/xgboost.joblib')
        self.scaler = load(f'{model_dir}/scaler.joblib')
        
        # Load metadata
        with open(f'{model_dir}/metadata.json', 'r') as f:
            meta = json.load(f)
            self.threshold = meta['threshold']
            self.features = np.array(meta['selected_features'])
            self.weights = meta['ensemble_weights']
        
        print(f"Model loaded - Threshold: {self.threshold:.3f}")
        print(f"Ensemble weights - Neural Net: {self.weights['neural_net']:.3f}, XGBoost: {self.weights['xgboost']:.3f}")
        print(f"Target: 95% accuracy + 70% sensitivity")
    
    def predict(self, raw_features):
        """Make balanced AFib prediction."""
        # Preprocess features
        if not isinstance(raw_features, np.ndarray):
            raw_features = np.array(raw_features).reshape(1, -1)
        
        # Handle missing values
        if pd.DataFrame(raw_features).isna().sum().sum() > 0:
            fill_values = getattr(self.scaler, 'center_', np.zeros(raw_features.shape[1]))
            raw_features = pd.DataFrame(raw_features).fillna(pd.Series(fill_values)).values
        
        # Scale and select features
        scaled = self.scaler.transform(raw_features)
        selected = scaled[:, self.features]
        
        # Neural network prediction
        nn_prob = self.neural_net.predict(selected, verbose=0)[0, 0]
        
        # XGBoost prediction
        xgb_prob = self.xgboost.predict_proba(selected)[0, 1]
        
        # Ensemble prediction
        ensemble_prob = (self.weights['neural_net'] * nn_prob + 
                        self.weights['xgboost'] * xgb_prob)
        
        prediction = 1 if ensemble_prob >= self.threshold else 0
        
        return {
            'prediction': 'AFib' if prediction == 1 else 'Normal',
            'probability': float(ensemble_prob),
            'confidence': float(max(ensemble_prob, 1 - ensemble_prob)),
            'threshold': float(self.threshold),
            'neural_net_prob': float(nn_prob),
            'xgboost_prob': float(xgb_prob)
        }

# =============================================================================
# Main Pipeline
# =============================================================================

def main():
    """Execute balanced AFib detection pipeline for 95% accuracy + 70% sensitivity target."""
    print("Balanced AFib Detection Pipeline")
    print("Target: 95% Accuracy + 70% Sensitivity")
    print("=" * 50)
    
    # Load and prepare data
    data = load_and_prepare_data()
    X_train, y_train = data['train']
    X_val, y_val = data['val']
    X_test, y_test = data['test']
    
    print(f"\nDataset prepared:")
    print(f"Train: {X_train.shape}, Validation: {X_val.shape}, Test: {X_test.shape}")
    
    # Train balanced ensemble
    print(f"\nTraining Balanced Ensemble (Neural Network + XGBoost)...")
    ensemble = BalancedEnsemble(X_train.shape[1])
    ensemble.fit(X_train, y_train, X_val, y_val)
    
    # Optimize threshold for balanced performance
    print(f"\nOptimizing threshold for balanced performance...")
    threshold = optimize_for_balanced_performance(ensemble, X_val, y_val)
    
    # Final evaluation on test set
    print(f"\nFinal evaluation on test set...")
    results = evaluate_balanced_model(ensemble, X_test, y_test, threshold)
    
    # Save balanced model
    save_balanced_model(ensemble, data['scaler'], data['features'], threshold, results)
    
    # Test deployment
    print(f"\nTesting deployment class...")
    detector = BalancedAFibDetector()
    sample_result = detector.predict(X_test[0])
    true_label = 'AFib' if y_test[0] == 1 else 'Normal'
    
    print(f"Sample prediction test:")
    print(f"True: {true_label}, Predicted: {sample_result['prediction']}")
    print(f"Ensemble probability: {sample_result['probability']:.3f}")
    print(f"Neural Net: {sample_result['neural_net_prob']:.3f}, XGBoost: {sample_result['xgboost_prob']:.3f}")
    
    print(f"\nPipeline Complete!")
    print(f"Final Accuracy: {results['accuracy']*100:.2f}%")
    print(f"Final Sensitivity: {results['sensitivity']*100:.1f}%")
    
    # Success assessment
    accuracy_target_met = results['accuracy'] >= 0.95
    sensitivity_target_met = results['sensitivity'] >= 0.70
    
    if accuracy_target_met and sensitivity_target_met:
        print("SUCCESS: Both balanced performance targets achieved!")
    elif results['accuracy'] >= 0.92 and results['sensitivity'] >= 0.65:
        print("VERY GOOD: Close to balanced performance targets")
    elif results['accuracy'] >= 0.90 and results['sensitivity'] >= 0.60:
        print("GOOD: Acceptable balanced performance achieved")
    else:
        print("IMPROVEMENT NEEDED: Consider further optimization")
    
    print(f"\nRecommended for clinical deployment: {results['accuracy'] >= 0.90 and results['sensitivity'] >= 0.60}")
    
    return results

if __name__ == "__main__":
    main()

Balanced AFib Detection Pipeline
Target: 95% Accuracy + 70% Sensitivity
Loading and preparing data for 99% accuracy target...
Original dataset: (5788, 14), AFib: 738, Normal: 5050
After outlier removal: (5209, 14), Removed: 579 samples
Advanced feature selection for maximum discrimination...
Consensus features selected: 14
Conservative class balancing for accuracy preservation...
Final balanced dataset - AFib: 2291, Normal: 3274

Dataset prepared:
Train: (5565, 14), Validation: (665, 14), Test: (782, 14)

Training Balanced Ensemble (Neural Network + XGBoost)...
Training Balanced Neural Network...


[I 2025-09-01 19:38:10,095] A new study created in memory with name: no-name-63b63a7b-3c1e-46ed-a770-7e40249e4af0


Training Balanced XGBoost...
Optimizing XGBoost for balanced performance...


[I 2025-09-01 19:38:10,434] Trial 0 finished with value: 0.9561701070301213 and parameters: {'n_estimators': 296, 'max_depth': 7, 'learning_rate': 0.13333878218723963, 'subsample': 0.8274269185566359, 'colsample_bytree': 0.9520818903700126, 'reg_alpha': 0.6364258315674661, 'reg_lambda': 1.1694524968132582, 'scale_pos_weight': 6.7664320053501665}. Best is trial 0 with value: 0.9561701070301213.
[I 2025-09-01 19:38:10,865] Trial 1 finished with value: 0.9590193353641141 and parameters: {'n_estimators': 288, 'max_depth': 8, 'learning_rate': 0.017646846840261653, 'subsample': 0.7552966508817806, 'colsample_bytree': 0.9278006994937493, 'reg_alpha': 0.23662839918890385, 'reg_lambda': 1.9040547870117248, 'scale_pos_weight': 3.02923084540173}. Best is trial 1 with value: 0.9590193353641141.
[I 2025-09-01 19:38:11,132] Trial 2 finished with value: 0.9504716503621355 and parameters: {'n_estimators': 375, 'max_depth': 5, 'learning_rate': 0.06802375301792594, 'subsample': 0.7658535312810106, 'cols

Best XGBoost balanced score: 0.9666
Optimizing ensemble weights for balanced performance...


[I 2025-09-01 19:38:29,796] A new study created in memory with name: no-name-c55b342f-0060-4f7f-8208-f3e7bef46721
[I 2025-09-01 19:38:29,819] Trial 0 finished with value: 0.9846623903700501 and parameters: {'weight_nn': 0.8386401501141227}. Best is trial 0 with value: 0.9846623903700501.
[I 2025-09-01 19:38:29,842] Trial 1 finished with value: 0.969466505922088 and parameters: {'weight_nn': 0.2438727823530754}. Best is trial 0 with value: 0.9846623903700501.
[I 2025-09-01 19:38:29,864] Trial 2 finished with value: 0.9827629048140548 and parameters: {'weight_nn': 0.875335911410622}. Best is trial 0 with value: 0.9846623903700501.
[I 2025-09-01 19:38:29,886] Trial 3 finished with value: 0.9641472101305897 and parameters: {'weight_nn': 0.151044775317692}. Best is trial 0 with value: 0.9846623903700501.
[I 2025-09-01 19:38:29,907] Trial 4 finished with value: 0.9846623903700501 and parameters: {'weight_nn': 0.8436603676605563}. Best is trial 0 with value: 0.9846623903700501.
[I 2025-09-01 

Optimal ensemble weights - Neural Net: 0.765, XGBoost: 0.235

Optimizing threshold for balanced performance...

Threshold analysis for balanced performance (95% accuracy + 70% sensitivity):
Threshold  Accuracy  Sensitivity  Specificity   FN   FP  Score
-----------------------------------------------------------------
    0.300    0.8842     0.7558      0.9033    21  56  0.958
    0.320    0.8842     0.7558      0.9033    21  56  0.958
    0.340    0.8872     0.7442      0.9085    22  53  0.960
    0.360    0.8887     0.7442      0.9102    22  52  0.961
    0.380    0.8887     0.7442      0.9102    22  52  0.961
    0.400    0.8962     0.7442      0.9188    22  47  0.966
    0.420    0.8977     0.7442      0.9206    22  46  0.967
    0.440    0.9008     0.7442      0.9240    22  44  0.969
    0.460    0.9038     0.7326      0.9292    23  41  0.971
    0.480    0.9038     0.7209      0.9309    24  40  0.971
    0.500    0.9038     0.7093      0.9326    25  39  0.971
    0.520    0.9053  

ValueError: Expected 2D array, got 1D array instead:
array=[ 0.22464673  0.16669841 -0.44067587 -0.50002896 -0.4889857  -0.84030769
 -0.44676907 -1.12390935 -0.64865749  0.38235294  0.         -0.416568
 -0.19820857  0.5625    ].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.